Escriba un segundo tokenizer que implemente los criterios del artículo de Grefenstette y Tapanainen para definir qué es una “palabra” (o término) y cómo tratar números y signos de puntuación. En este  caso  su  tokenizer deberá extraer y tratar como un único término: 

Abreviaturas tal cual están escritas (por ejemplo, Dr., Lic., S.A., etc.)
Direcciones de correo electrónico y URLs.
Números (por ejemplo, cantidades, teléfonos).
Nombres propios (por ejemplo, Villa Carlos Paz, Manuel Belgrano, etc.)
 
Utilice la colección para debugging de expresiones regulares provista por el equipo docente para extraer y comparar  la salida de su programa con los metadatos de la colección tal como lo realizó en el punto 1.

Por último, extraiga y almacene la misma información que en el punto 2 sobre la colección RI-tknz-data utilizando su nuevo tokenizer.

In [ ]:
import os
import re


def leer_archivos(carpeta):
    documentos = {}

    for archivo in os.listdir(carpeta):
        match = re.search(r"\d+", archivo)  # Extraer el numero
        doc_id = int(match.group())
        with open(os.path.join(carpeta, archivo), "r", encoding="utf-8") as f:
            documentos[doc_id] = f.read()

    return documentos

def tokenizer(texto):
    # Extraer terminos especiales
    urls = re.findall(r"(?:https?|ftp)://[^\s]+", texto)
    emails = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", texto) # usuario@dominio.com
    numeros = re.findall(r"\(?\d{2,4}\)?[-.\s]?\d{2,4}[-.\s]?\d{2,4}", texto) # (012) 345-6789
    nombres_propios = re.findall(r"[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+(?:\s[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)+", texto) # Juan Pérez
    acronimos = re.findall(r"\b(?:[A-Z]+\.){2,}", texto) # NO FUNCIONA 
    abreviaturas = re.findall(r"\b(?:[A-ZÁÉÍÓÚÑ][a-záéíóúñ]{1,4}\.)", texto) # NO FUNCIONA


    # Eliminar esos términos del texto para no procesarlos dos veces
    
    for termino in urls + emails + numeros + nombres_propios + abreviaturas + acronimos:
        texto = texto.replace(termino, "")

    # Eliminar caracteres especiales
    texto = re.sub(r"[^a-zA-ZÁÉÍÓÚÑáéíóúñ0-9\s]", "", texto)
    # Convertir a minusculas
    texto = texto.lower()
    # Tokenizar (separar por espacios)
    tokens = texto.split()

    # Restaurar los términos especiales en la lista final de tokens
    tokens.extend(urls + emails + numeros + nombres_propios + abreviaturas + acronimos)

    return tokens

def procesar(texto, doc_id):

    global nTokens
    global datos

    tokens = tokenizer(texto)
    nTokens += len(tokens)
    for token in tokens:
        if token not in datos:
            datos[token] = {
                "docsId": [],
                "df": 0,
            }  # Inicializo la estructura de cada termino

        if doc_id not in datos[token]["docsId"]:
            datos[token]["docsId"].append(doc_id)
            datos[token]["df"] = len(datos[token]["docsId"])


def analisis_lexico(carpeta):
    global nDocumentos
    documentos = leer_archivos(carpeta)
    for doc_id, contenido in documentos.items():
        procesar(contenido, doc_id)
        nDocumentos += 1


datos = {}
nTokens = 0
nDocumentos = 0
path = "RE_collection_test/collection_test_ER2"

analisis_lexico(path)

- Lista de términos y su DF

In [64]:
for termino, info in datos.items():
    print(f"{termino}: {info['df']}")

lluvia: 320
tenis: 130
https://www.youtube.com/watch?v=zKqUfwB84Zg: 60
https://www.researchgate.net/publication/2360239: 1000
2021-12-09: 132
2360239: 998
Domingo Faustino Sarmiento: 990
https://docs.google.com/document/d/1ninD55Cfbb_7PksDirN0XghzNHJZ_N93lheQzF1aOZY/edit?usp=sharing: 120
abc.def@mail-archive.com: 90
15413355: 220
2360239 1541: 215
3355 15413355: 197
Sra.: 80
ternero: 400
3355: 46
15413355 1541: 166
http://www.tcpipguide.com/free/t_TCPIPProcessesMultiplexingandClientServerApplicati.htm: 110
i: 150
D.A.S.M.: 150
EE.UU.: 70
20211209: 23
botella: 200
2360239 2021: 136
12-09 2021: 96
mratto@mail.unlu.edu.ar: 100
https://www.labredes.unlu.edu.ar/tyr2022: 50
2022 15413355: 10
a: 30
U.S.: 30
domingo: 9
faustino: 9
sarmiento: 9
Zg Domingo Faustino Sarmiento: 9
ftp://unlu.edu.ar: 40
1541: 45
casa: 20
2022 2021-12: 5
09: 1
154109: 2
3355 2021-12: 20
09 2021-12: 8
09 15413355: 9
12-09 1541: 19
15413355 2021: 20
Domingo Faustino Sarmiento Sra: 1
1209: 2
20213355: 2
202109: 1


- Cantidad de tokens

In [65]:
print(nTokens)

111688


- Cantidad de términos 

In [66]:
nTerminos = len(list(datos.keys()))
print(nTerminos)

48


- Cantidad de documentos procesados  

In [67]:
print(nDocumentos)

1000


Tests

In [68]:
import json

with open("RE_collection_test/collection_data.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)
    for term in test_data['data']:
        print(term['term'] + " : " + str(term['df']))
        

casa : 20
U.S.A : 30
ftp://unlu.edu.ar : 40
https://www.labredes.unlu.edu.ar/tyr2022 : 50
https://www.youtube.com/watch?v=zKqUfwB84Zg : 60
EE.UU. : 70
Sra. : 80
abc.def@mail-archive.com : 90
mratto@mail.unlu.edu.ar : 100
http://www.tcpipguide.com/free/t_TCPIPProcessesMultiplexingandClientServerApplicati.htm : 110
https://docs.google.com/document/d/1ninD55Cfbb_7PksDirN0XghzNHJZ_N93lheQzF1aOZY/edit?usp=sharing : 120
tenis : 130
2021-12-09 : 140
D.A.S.M.I : 150
botella : 200
ternero : 400
15413355 : 220
Domingo Faustino Sarmiento : 1000
lluvia : 320
https://www.researchgate.net/publication/2360239 : 1000
